In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_columns',40)
from surprise import Reader,Dataset,SVD
from surprise.model_selection import cross_validate
from surprise import KNNWithMeans, KNNBaseline
from surprise.model_selection import RandomizedSearchCV, GridSearchCV

# Load Data

In [2]:
movies_rating=pd.read_csv('Data/movies_rating.csv')
movies_data=pd.read_csv('Data/movies_data(copy1).csv')
movies_final=pd.read_csv('Data/movies_data.csv')

In [46]:
movies_data.

,movieId,tmdbId,title,runtime,cast,imdb_link,poster,year,genre,director,soup,language,genre_name,language_name,runtime_name,score,rating_2
0,318,278,The Shawshank Redemption,142,Tim Robbins,http://www.imdb.com/title/tt111161,https://image.tmdb.org/t/p/w500//q6y0Go1tsGEsm...,1994,5,Frank Darabont,prison corruption policebrutality timrobbins m...,1,Drama,English,2h 22min,8.295433,4.9
1,858,238,The Godfather,175,Marlon Brando,http://www.imdb.com/title/tt68646,https://image.tmdb.org/t/p/w500//3bhkrj58Vtu7e...,1972,5,Francis Ford Coppola,italy loveatfirstsight lossoffather marlonbran...,1,Drama,English,2h 55min,8.138164,4.7
2,2959,550,Fight Club,139,Edward Norton,http://www.imdb.com/title/tt137523,https://image.tmdb.org/t/p/w500//bptfVGEQuv6vD...,1999,5,David Fincher,supportgroup dualidentity edwardnorton bradpit...,1,Drama,English,2h 19min,8.136253,4.6
3,296,680,Pulp Fiction,154,John Travolta,http://www.imdb.com/title/tt110912,https://image.tmdb.org/t/p/w500//dRZpdpKLgN9nk...,1994,6,Quentin Tarantino,transporter brothel drugdealer johntravolta sa...,1,Others,English,2h 34min,8.118890,4.9
4,58559,155,The Dark Knight,152,Christian Bale,http://www.imdb.com/title/tt468569,https://image.tmdb.org/t/p/w500//qJ2tW6WMUDux9...,2008,5,Christopher Nolan,dccomics crimefighter secretidentity christian...,1,Drama,English,2h 32min,8.075906,4.5
5,356,13,Forrest Gump,142,Tom Hanks,http://www.imdb.com/title/tt109830,https://image.tmdb.org/t/p/w500//h5J4W4veyxMXD...,1994,3,Robert Zemeckis,vietnamveteran hippie mentallydisabled tomhank...,1,Comedy,English,2h 22min,8.018367,4.5
6,79132,27205,Inception,148,Leonardo DiCaprio,http://www.imdb.com/title/tt1375666,https://image.tmdb.org/t/p/w500//9gk7adHYeDvHk...,2010,1,Christopher Nolan,lossoflover dream kidnapping leonardodicaprio ...,1,Action,English,2h 28min,7.996662,4.3
7,527,424,Schindler's List,195,Liam Neeson,http://www.imdb.com/title/tt108052,https://image.tmdb.org/t/p/w500//c8Ass7acuOe4z...,1993,5,Steven Spielberg,factory concentrationcamp hero liamneeson benk...,1,Drama,English,3h 15min,7.975862,4.7
8,109487,157336,Interstellar,169,Matthew McConaughey,http://www.imdb.com/title/tt816692,https://image.tmdb.org/t/p/w500//gEU2QniE6E77N...,2014,6,Christopher Nolan,savingtheworld artificialintelligence fatherso...,1,Others,English,2h 49min,7.971242,4.5
9,112552,244786,Whiplash,105,Miles Teller,http://www.imdb.com/title/tt2582802,https://image.tmdb.org/t/p/w500//6uSPcdGNA2A6v...,2014,5,Damien Chazelle,jazz obsession milesteller j.k.simmons melissa...,1,Drama,English,1h 45min,7.971110,4.4


# Content-based filtering (credits, genres and keywords)

In [ ]:
# # use CountVectorizer() because we do not want to down-weight the presence of an actor/director if he or she has acted or directed in relatively more movies.
# from sklearn.feature_extraction.text import CountVectorizer
# count=CountVectorizer(stop_words='english')
# count_matrix=count.fit_transform(movies_data['soup'])

In [ ]:
# # compute the cosine similartity matrix based on the count_matrix
# from sklearn.metrics.pairwise import cosine_similarity
# cosine_sim2=cosine_similarity(count_matrix,count_matrix)

# Collaborative filtering
* user based filtering - issue: users tastes change
* item based filtering - issue: sparsity and scalability
* single value decomposition - capture the similarity between users and items, use `Root Mean Square Error (RMSE)`.

## Data Exploration
* 671 users (median:30 ratings)(1.87% of users only rated one movie; 4% of users rated less than 4 movies)
* 45115 movies (17% of movies only have one rating; 35% of movies received less than 4 ratings)
* 3.53 mean rating

In [ ]:
# rating distribution
sns.displot(data=movies_rating,x='rating',aspect=1)

## Model and Prediction

In [3]:
reader=Reader()
data = Dataset.load_from_df(movies_rating[['userId', 'movieId', 'rating']], reader)

In [4]:
index=movies_rating[(movies_rating['userId']==2)&(movies_rating['rating']==5)].movieId.values
for i in index:
     print(movies_data[movies_data['movieId']==i].title)

96    Ex Machina
Name: title, dtype: object
78    V for Vendetta
Name: title, dtype: object
66    Good Will Hunting
Name: title, dtype: object
68    Harry Potter and the Prisoner of Azkaban
Name: title, dtype: object
64    Up
Name: title, dtype: object
60    American Beauty
Name: title, dtype: object
98    Toy Story 3
Name: title, dtype: object
55    Big Hero 6
Name: title, dtype: object
53    WALL·E
Name: title, dtype: object
144    It's a Wonderful Life
Name: title, dtype: object
100    A Beautiful Mind
Name: title, dtype: object
131    Ratatouille
Name: title, dtype: object
115    The Pursuit of Happyness
Name: title, dtype: object
127    Deadpool
Name: title, dtype: object


### Tuning
* KNN: 0.9118108079535345 {'sim_options': {'name': 'msd', 'user_based': True, 'min_support': 3}, 'min_k': 3}
* KNN: 0.9264617753608795 {'sim_options': {'name': 'cosine', 'user_based': False, 'min_support': 2}, 'min_k': 2}
* KNN: 0.9160193344530845 {'sim_options': {'name': 'pearson', 'user_based': True, 'min_support': 3}, 'min_k': 3}
* SVD: 0.893082788353421  {'n_factors': 90, 'n_epochs': 26, 'lr_all': 0.0041, 'reg_all': 0.035}

In [ ]:
# Tuning knn
sim_options={'name':['pearson'],
            'user_based':[False,True],
            'min_support':[1,2,3,4,5]
            }
param_grid={'sim_options':sim_options,'min_k':[1,2,3]}
gs=RandomizedSearchCV(KNNWithMeans,param_grid,measures=['rmse'],cv=4)
gs.fit(data)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

In [ ]:
# Tuning SVD
param_grid={'n_factors':[80,85,90,95],
            'n_epochs':[22,24,26,29],
            'lr_all':[,0.0041,0.0043,0.0045],
           'reg_all':[0.028,0.035,0.0037,0.004]}
gs=RandomizedSearchCV(SVD,param_grid,measures=['rmse'],cv=4)
gs.fit(data)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

In [37]:
# KNN
sim_options={'name':'cosine',
            'user_based':True,
            'min_support':2}
# bsl_options={'method':'als'
#             }
# algo_knn=KNNBaseline(sim_options=sim_options,bsl_options=bsl_options)
algo_knn=KNNWithMeans(sim_options=sim_options,min_k=2)
# cross_validate(algo_knn,data,measures=['RMSE','MAE'],cv=4,verbose=True)

In [6]:
# SVD
# param_grid={'n_factors':85,
#             'n_epochs':24,
#             'lr_all':0.004,
#            'reg_all':0.035}
algo_svd=SVD(n_factors=90,
        n_epochs=26,
        lr_all=0.004,
        reg_all=0.035)
# cross_validate(algo_svd,data,measures=['RMSE','MAE'],cv=4,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 4 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Mean    Std     
RMSE (testset)    0.8872  0.8940  0.8938  0.9009  0.8940  0.0048  
MAE (testset)     0.6826  0.6889  0.6912  0.6928  0.6889  0.0039  
Fit time          9.16    7.51    7.40    7.22    7.83    0.78    
Test time         0.24    0.22    0.29    0.28    0.26    0.03    


{'test_rmse': array([0.88716477, 0.8940118 , 0.89376812, 0.90087879]),
 'test_mae': array([0.68264471, 0.68889274, 0.69117915, 0.6928164 ]),
 'fit_time': (9.164883852005005,
  7.508480072021484,
  7.404152154922485,
  7.223479986190796),
 'test_time': (0.2353501319885254,
  0.21896004676818848,
  0.29210424423217773,
  0.28119778633117676)}

In [16]:
trainset=data.build_full_trainset()

In [17]:
algo_svd.fit(trainset)

In [40]:
algo_knn.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [41]:
# Predict movie ratings based on users' preferences
def predict_rating(userId,movieId,model):
    rating_s=model.predict(userId,movieId)
    rating_s=round(rating_s.est,1)
    return rating_s

In [42]:
movies_data['rating_2']=movies_data.movieId.apply(lambda s: predict_rating(2,s,algo_knn))

In [ ]:
# Updata movies_data
movies_data['rating_1']=movies_data.movieId.apply(lambda s: predict_rating(1,s,algo_knn))

In [ ]:
movies_final=movies_data[['movieId','title','runtime','poster','year','genre','director','language','genre_name','language_name','runtime_name'
                         ,'rating_1','rating_2','cast']]

In [ ]:
# Save file
# movies_final.to_csv('movies_data.csv',index=False)